In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from requests_testadapter import Resp
import requests
#from requests_file import FileAdapter
import os.path


In [7]:
#limpiamos el DF en memoria para iniciar el proceso.
df = pd.DataFrame()
df_reviews = pd.DataFrame()

#ruta de donde vamos a tomar los htmls.
ruta = "Data/Resto2/"
 
#recorremos todos los archivos en la ruta. 
for filename in os.listdir(ruta):
        file = open(ruta+filename, "r",  encoding='ascii') #abrimos el archivo

        bsObj = BeautifulSoup(file, 'html.parser') #generamos el objeto beautiful soup
    

#TOMA DE CONTENIDO - ETIQUETAS HTML
                       
        aclass = bsObj.findAll("a", {"class":"display-extended-links"}) #obtenemos barrio y tipo comida de esta var
        
        tipo_comida_pre = bsObj.findAll("a", {"class":"display-extended-links"}) 
        tipo_comida = []                      
        if len(tipo_comida_pre) - 2 >= 0:
            for i in range( 0, len(tipo_comida_pre) - 2 ):
                tipo_comida.append(tipo_comida_pre[i].get_text())     
        
        
        actividades_pre = bsObj.findAll("h4",{"class":"tile-text-sm description"})
        actividades= []
        for i in range( 0, len(actividades_pre)):
            actividades.append(actividades_pre[i].get_text())
        
        
        extras_pre = bsObj.findAll("h4",{"class":"title-text-sm description"})
        extras = []
        for i in range( 0, len(extras_pre)):
            extras.append(extras_pre[i].get_text())
        
        barrio = []
        if len(aclass) -2 >= 0 :
            barrio = bsObj.findAll("a", {"class":"display-extended-links"})[len(aclass)-2].text 

        nombre_lugar = filename[0:len(filename)-5] 

        
        price_range = []
        #price_range = bsObj.find("span", {"class":"price-range"}) - version vieja borrar
        price_range_b = bsObj.find("span", {"class":"price-range"})
        if price_range_b != None:
            price_range = price_range_b['class'][1].replace("price-range-","")
 
    
        reviews = bsObj.findAll("p",{"class":"activity-text"})
        reviews_date = bsObj.findAll("div",{"class":"activity-date"})
        
        
        #en proceso
        puntaje_review = []      
        puntaje_review = bsObj.findAll("div",{"class":"activity-ratings"})
#        puntaje_review_b = puntaje_review[0].findAll("span")
#         review_comida_b = puntaje_review_b[2].get_text().strip()
#         review_servicio_b = puntaje_review_b[5].text
#         review_ambiente_b = puntaje_review_b[8].text
#         review_comida = " ".join(review_comida_b.split())
#         review_servicio = " ".joinreview_servicio_b.split())
#         review_ambiente = " ".join(review_ambiente_b.split())
        
        
        
        #GUARDADO DE DATOS CON APPEND MEDIANTE ITERACIONES
        
        serie = pd.Series([nombre_lugar, tipo_comida, barrio, price_range, actividades, extras], index=['nombre_lugar', 'tipo_comida', 'barrio', 'price_range', 'actividades', 'extras'])
        df = df.append(serie, ignore_index=True)
        
        serie_reviews = pd.Series([nombre_lugar, reviews, reviews_date, puntaje_review]) #index=['nombre_lugar', 'reviews'])
        df_reviews = df_reviews.append(serie_reviews, ignore_index=True)

        

export_csv = df.to_csv ('df_scrapping_nuevo.csv', index = None, header=True, encoding='ascii') 


In [8]:
#tomamos df_reviews generado en el paso anterior e iteramos por la lista de reviews y sus fechas y las guardamos como registros
#independientes en un nuevo dataframe.

df_reviews_shaped = pd.DataFrame()
resto_counter = 0
for resto in df_reviews[0].values:
    counter2 = 0
    for item in df_reviews.iloc[resto_counter][1]:
        review = df_reviews.iloc[resto_counter][1][counter2].text
        
        fecha = df_reviews.iloc[resto_counter][2][counter2].text
        
        #iteramos dentro de puntaje y la descomponemos en columna comida, servicio y ambiente.
        #agregamos puntaje al DF reviews, por si queremos controlar la info, despues lo dropeamos.
        try:
            puntaje = df_reviews.iloc[resto_counter][3][counter2]
            puntaje = puntaje.findAll("span")

            comida = puntaje[2].get_text()
            comida = comida.replace('\\t','').replace('\\n','').strip()

            servicio = puntaje[5].get_text()
            servicio = servicio.replace('\\t','').replace('\\n','').strip()

            ambiente = puntaje[8].get_text()
            ambiente = ambiente.replace('\\t','').replace('\\n','').strip()
            
        except IndexError:
             puntaje = None
             comida = None
             servicio = None
             ambiente = None
        
        
        
        serie_reviews_2 = pd.Series([resto, review, fecha, puntaje, comida, servicio, ambiente], index=['resto', 'review', 'fecha', 'puntaje', 'comida', 'servicio', 'ambiente'])
        df_reviews_shaped = df_reviews_shaped.append(serie_reviews_2, ignore_index=True)
        counter2=counter2 + 1
    resto_counter = resto_counter + 1
    
    df_reviews_shaped = df_reviews_shaped[['resto','fecha','review','comida', 'servicio', 'ambiente']]
        
#export_csv = df_reviews.to_csv (r'C:/Users/QQ-Haum/Documents/desafio-final/df_reviewss.csv', index = None, header=True) 
#export_csv = df_reviewss.to_csv (r'C:/Users/QQ-Haum/Documents/desafio-final/df_reviewss.csv', index = None, header=True) 
export_csv = df_reviews_shaped.to_csv ('df_scrapping_reviews_nuevoo.csv', index = None, header=True, encoding='ascii') 



In [10]:
df_reviews_shaped.review


0    Fui con mi esposa(dominicana ella)y la verdad ...
1    Buena experiencia, ideal para ir con amigos. B...
2    Rinc\xc3\xb3n Patag\xc3\xb3nico es un cl\xc3\x...
3    Fuimos previa reserva ya que el local es chico...
4    Fui con mis amigas, la pasamos muy bien!!!\r\n...
5    Fui con mis amigas, la pasamos muy bien!!!\r\n...
6    Un lugar super recomendable. con platos exquis...
7    Un lugar super recomendable. con platos exquis...
8    Excelente lugar..intimo..para disfrutar de una...
Name: review, dtype: object